In [1]:
!pip install openai
!pip install numpy pandas matplotlib
!pip install rpy2
!pip install groqflow
!pip install transformers
!pip install groq

INFO: pip is looking at multiple versions of onnxconverter-common to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 9.2 MB/s eta 0:00:00


In [2]:
%load_ext rpy2.ipython


In [3]:
import openai

from google.colab import userdata


openai_api_key = userdata.get('OPENAI_API_KEY')


In [4]:
from groq import Groq

groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq client
client = Groq(api_key=groq_api_key)

# Define the messages for the LLM
messages = [
    {"role": "system", "content": "You are an expert psychometrician creating test items."},
    {"role": "user", "content": "Generate two items for the construct of Rigid Perfectionism: one regular-keyed and one reverse-keyed. Make them concise and clear."}
]

# Create the completion request
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

# Process the response
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


Here are two test items for the construct of Rigid Perfectionism:

1. Regular-keyed item:

"I often feel a sense of anxiety or distress if my work or performance does not meet my exceptionally high standards."

(Rating scale: 1 = Strongly disagree, 2 = Somewhat disagree, 3 = Neither agree nor disagree, 4 = Somewhat agree, 5 = Strongly agree)

2. Reverse-keyed item:

"I am able to perform well even when I don't meet my own expectations for quality."

(Rating scale: 1 = Strongly disagree, 2 = Somewhat disagree, 3 = Neither agree nor disagree, 4 = Somewhat agree, 5 = Strongly agree)

In the regular-keyed item, the stem induces respondents to endorse the item if they experience anxiety or distress when their performance doesn't meet their high standards, which is a characteristic of rigid perfectionism. In the reverse-keyed item, the stem suggests that the respondent is able to perform well even when they don't meet their own expectations, which is the opposite of rigid perfectionism. By i

In [5]:
items = [
    {
        "item": "I often feel anxious when I make a mistake, even if it's a minor one, and I worry that it will affect my overall performance.",
        "type": "regular-keyed"
    },
    {
        "item": "I am comfortable with the idea that I may not be perfect all the time, and I don't get too upset when I make a mistake or things don't go exactly as planned.",
        "type": "reverse-keyed"
    }
]

import pandas as pd
items_df = pd.DataFrame(items)
items_df.to_csv("rigid_perfectionism_items.csv", index=False)


In [6]:
import requests
import numpy as np

embedding_endpoint = "https://api.openai.com/v1/embeddings"
embedding_model = "text-embedding-3-small"

headers = {
    "Authorization": f"Bearer {openai_api_key}",
    "Content-Type": "application/json"
}

embeddings = []
for item in items_df["item"]:
    response = requests.post(
        embedding_endpoint,
        headers=headers,
        json={"model": embedding_model, "input": item}
    )

    # Debugging: Print the full response
    print(response.json())

    if 'data' in response.json():
        embeddings.append(response.json()["data"])
    else:
        print(f"Error: {response.json().get('error', {}).get('message', 'Unknown error')}")

# Convert embeddings into a usable format
embeddings_array = np.array(embeddings)

{'object': 'list', 'data': [{'object': 'embedding', 'index': 0, 'embedding': [0.024861827, -0.021199256, 0.02111436, 0.039924454, -0.0067733317, 0.017221363, 0.039439347, 0.019028394, -0.05782497, 0.0584071, -0.008392382, -0.008519724, -0.0011043806, 0.0016357263, 0.0012529452, 0.03856615, 0.0013962038, -0.004866248, 0.02532268, 0.009799198, 0.034175918, 0.020156272, 0.024025014, 0.009301961, 0.042374257, -0.029397594, 0.01702732, -0.01622689, 0.049068756, 0.034515493, 0.02665673, -0.019343713, -0.029712915, -0.04222872, 0.015329438, 0.017936898, 0.029300572, -0.0066035436, 0.015438588, -0.017609451, -0.037110824, 0.040142756, 0.032211225, -0.02505587, 0.037013803, -0.040045734, -0.017075831, 0.008234723, -0.0121762315, 0.02862142, -0.052343242, 0.06306414, 0.007883018, 0.091734074, 0.030367812, -0.007373654, 0.0031956544, -0.0067733317, 0.013255598, 0.0155841205, 0.0329874, 0.011921549, 0.002904589, 0.012903894, -0.019343713, -0.0102843065, 0.030319301, -0.025468213, 0.012061018, 0.00

In [9]:
# Change this cell to R
%%R

# Install devtools if not already installed
if (!"devtools" %in% rownames(installed.packages())) {
    install.packages("devtools")
}

# Load devtools library
library(devtools)

# Install EGA from GitHub
install_github("hfgolino/EGA")

# Load EGA library
library(EGA)



(as ‘lib’ is unspecified)















































	‘/tmp/RtmpIPEjlc/downloaded_packages’



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpIPEjlc/remotesfa485b1761/hfgolino-EGAnet-18e50c3/DESCRIPTION’ ... OK
* preparing ‘EGAnet’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘EGAnet_2.1.0.tar.gz’



(as ‘lib’ is unspecified)





 



 


 

 




Error in library(EGA) : there is no package called ‘EGA’


In [8]:
%R -i embeddings_array

%%R
library(EGAnet)

# Perform Exploratory Graph Analysis (EGA)
ega_result <- EGA(as.matrix(embeddings_array))
print(ega_result)


SyntaxError: invalid syntax (<ipython-input-8-5a8d46495c34>, line 7)